In [1]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import time
from tqdm import tqdm  # For progress bars

In [7]:


# Set paths
BASE_DIR = "/content/drive/MyDrive/Info_assurance/Dawn"
WEATHER_FOLDERS = ["Snow/images", "Sand/images", "Rain/images", "Fog/images"]
MODEL_PATH = "/content/drive/MyDrive/Info_assurance/models/CNN-PP_models/train100_efficientnet_filter_predictor.keras"
OUTPUT_DIR = "/content/drive/MyDrive/Info_assurance/predictions"

# Model input shape - must match what the model was trained with
INPUT_SHAPE = (200, 200, 3)  # Adjust to match your trained model

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Enable unsafe deserialization
tf.keras.config.enable_unsafe_deserialization()

# Function to preprocess an image for prediction
def preprocess_image(img_path, input_shape):
    """Preprocess an image for model prediction"""
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error reading image: {img_path}")
        return None

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (input_shape[1], input_shape[0]))
    img = img.astype(np.float32) / 255.0
    return np.expand_dims(img, axis=0)  # Add batch dimension

# Load the trained model
print("Loading trained model...")
model = tf.keras.models.load_model(MODEL_PATH)
print("Model loaded successfully")

# Process each weather folder
for weather in WEATHER_FOLDERS:
    weather_dir = os.path.join(BASE_DIR, weather)

    if not os.path.exists(weather_dir):
        print(f"Warning: {weather_dir} does not exist. Skipping...")
        continue

    print(f"\nProcessing {weather} images...")

    # Get all image files in the folder
    image_files = []
    for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']:
        image_files.extend([f for f in os.listdir(weather_dir) if f.endswith(ext)])

    if not image_files:
        print(f"No images found in {weather_dir}. Skipping...")
        continue

    print(f"Found {len(image_files)} images in {weather} folder")

    # Initialize list to store predictions
    predictions = []

    # Process each image
    for img_file in tqdm(image_files, desc=f"Predicting {weather}"):
        img_path = os.path.join(weather_dir, img_file)

        # Preprocess image
        img = preprocess_image(img_path, INPUT_SHAPE)
        if img is None:
            print(f"Skipping {img_file} due to preprocessing error")
            continue

        # Make prediction
        wb_pred, gamma_pred, tone_pred, contrast_pred, usm_pred, fog_pred = model.predict(img, verbose=0)

        # Store predictions in a dictionary
        pred = {
            'filename': img_file,
            'wb_r': float(wb_pred[0, 0]),
            'wb_g': float(wb_pred[0, 1]),
            'wb_b': float(wb_pred[0, 2]),
            'gamma': float(gamma_pred[0, 0]),
            'contrast': float(contrast_pred[0, 0]),
            'usm': float(usm_pred[0, 0]),
            'fog_param': float(fog_pred[0, 0]),
        }

        # Add tone predictions
        for j in range(8):
            pred[f'tone_{j}'] = float(tone_pred[0, j])

        predictions.append(pred)

    # Convert to DataFrame
    pred_df = pd.DataFrame(predictions)

    # Save to CSV
    output_file = os.path.join(OUTPUT_DIR, f"{weather}_predictions.csv")
    pred_df.to_csv(output_file, index=False)
    print(f"Saved predictions for {len(pred_df)} {weather} images to {output_file}")

print("\nPrediction process completed for all weather conditions!")

Loading trained model...
Model loaded successfully

Processing Snow/images images...
Found 204 images in Snow/images folder


Predicting Snow/images: 100%|██████████| 204/204 [01:24<00:00,  2.41it/s]


Saved predictions for 204 Snow/images images to /content/drive/MyDrive/Info_assurance/predictions/Snow/images_predictions.csv

Processing Sand/images images...
Found 323 images in Sand/images folder


Predicting Sand/images: 100%|██████████| 323/323 [02:11<00:00,  2.46it/s]


Saved predictions for 323 Sand/images images to /content/drive/MyDrive/Info_assurance/predictions/Sand/images_predictions.csv

Processing Rain/images images...
Found 200 images in Rain/images folder


Predicting Rain/images: 100%|██████████| 200/200 [01:23<00:00,  2.38it/s]


Saved predictions for 200 Rain/images images to /content/drive/MyDrive/Info_assurance/predictions/Rain/images_predictions.csv

Processing Fog/images images...
Found 300 images in Fog/images folder


Predicting Fog/images: 100%|██████████| 300/300 [02:00<00:00,  2.49it/s]


Saved predictions for 300 Fog/images images to /content/drive/MyDrive/Info_assurance/predictions/Fog/images_predictions.csv

Prediction process completed for all weather conditions!
